# Exercise #3: Battleships (2/2)

### Probabilistic Machine Learning

- **Lecturer**: Prof. Philipp Hennig
- **Term**: SoSe 2020
- **Due Date**: Monday, 11 May 2020


![battleship rules](https://upload.wikimedia.org/wikipedia/commons/e/e4/Battleships_Paper_Game.svg)

In the previous exercise sheet on the game _battleships_, we studied prior and posterior probabilities of getting a hit by enumerating all possible locations of one boat. We established that it is computationally unfeasible to obtain the full posterior by enumeration for the number of ships in the rules.

Hence, to render the problem tractable, we need to approximate the posterior over hits given hit/miss observations. We achieve this through _Monte Carlo_ sampling. Instead of enumerating _all_ board states, we randomly sample `nb_samples` valid board states (i.e. configurations of ship arrangements on the board) and use their sum to estimate the posterior.

__The goal of this assignment is to write such a  Monte Carlo sampler.__

### __3.1__ Getting started

There are a few accompanying files that are required for the game. You will only need to alter `MC_agent.py`, all other vital routines are provided.
- `main.py`: Run this script to play the game. You can select which players should play against each other: `"human"`, `"random"`, or `"MC"`. The first two are already implemented, the latter is our task here.
- `board.py`: contains a class to track the current state of the board (observations), and to save the placement of the ships.  
- `game.py`: the game itself with relevant methods (initialize the game, calling the respective agents for their moves, checking if the game is over, etc.)
- `human_agent.py`: The human agent (interactive input)
- `random_agent.py`: The random agent (just selects a previously unobserved location at random)
- `MC_Agent`: this is where you should implement the Monte Carlo Agent for the virtual opponent. Initially, it randomly selects a query location (i.e. you can play the game by running `main.py`, but the computer makes uninformed moves.

In the current state, you can already play the game against a random agent, or have two random agents play against each other.

In [1]:
from main import play_battleships
import numpy as np
%matplotlib inline

In [2]:
play_battleships('random', 'random')


Player1's observations   Player2's observations

  0 1 2 3 4 5 6 7 8 9      0 1 2 3 4 5 6 7 8 9
0 . . X . . . . X . .    0 . . . . X - . . . .
1 . . X . . . . X . .    1 . . . . X X . . - .
2 . X X X X . . X . .    2 . . . . . . . . . .
3 . . - . . . . . X .    3 . . . - . . . . X .
4 X . . . . . . . X .    4 - . . . . . . X - .
5 . - X X X X X . X .    5 . . . . . . . X - .
6 - . - . . . . . . .    6 . . . . X X . X X .
7 . . . . . - . . - .    7 . - . . . . . X . .
8 . . . . . . . - . .    8 . . . . X X X X . X
9 . . . . . . . . . -    9 . . - . . . . . . .
Game over! Player 1 won!


True

### __3.2__ Implement the Monte Carlo Agent

Your task is to replace the method `select_observation` in the class `MCAgent` by a routine that samples board states to estimate the posterior probability of getting a hit, and then choosing the location that maximizes this probability.

The state of observations is saved in the variable `self.observed_board`. A `0` is an unobserved location, `-1` a miss or a sunk ship, and a `1` a hit of a ship that is not yet sunk.

In `select_observation`, we first define a variable `score_board` through which we will assign scores to each field. Therefore it has the shape of the `self.observed_board`.

_The key idea of the Monte Carlo agent is to sum up many possible boat placements and choose the location with the largest score (most boats placed) as new query location._

Here is how to proceed (see comments in the function):
1. Check if there is an "open" hit, i.e. a hit that does not belong to a sunk ship. If so, reduce the number of samples (e.g. to a tenth) and force possible boats to pass through the hit(s)
2. In case there is no open hit, simulate `nb_samples` boats, the location and orientation of which are chosen at random. Track them using the `score_board`.
3. Find the location with the largest score and return its indices.

Once you're done, you can play against your agent:

In [5]:
# play_battleships('MC', 'human')

n_games = 30

counter = {
    True: 0,
    False: 0,
    'draw': 0
}

for _ in range(n_games):
    winner = play_battleships('random', 'MC')
    if winner is None:
        counter['draw'] += 1
    else:
        counter[winner] += 1


Player1's observations   Player2's observations

  0 1 2 3 4 5 6 7 8 9      0 1 2 3 4 5 6 7 8 9
0 . X X X . X . . - .    0 . . . . . . . . . .
1 . . . . . X . . . .    1 . . . . . . . . . .
2 . . . . . . . . - .    2 . . . . . . . . . .
3 . . . . . . . . . .    3 X . . . . . . . . .
4 . - . . . . . . . .    4 X . . . . . X . . .
5 X X X . . . . . . .    5 X X X . . . . X . .
6 . . . . . . . . X .    6 X X X . . . . X . .
7 . . - - X X . . . .    7 X X X . . . . . . .
8 . . . - . . . . . .    8 . . X . . . . X . .
9 . . . X X X X X . .    9 . . X X - - - - - -
Game over! Player 2 won!


In [7]:
print(f"Random player won {counter[1]} / {n_games} ({counter[1] / n_games * 100:.2f}%)")
print(f"MC player won {counter[0]} / {n_games} ({counter[0] / n_games * 100:.2f}%)")
print(f"Number of draws {counter['draw']} / {n_games} ({counter['draw'] / n_games * 100:.2f}%)")

Random player won 6 / 30 (20.00%)
MC player won 23 / 30 (76.67%)
Number of draws 1 / 30 (3.33%)


### __3.2__ Further analysis
__Task 1:__  
Surface the score matrix of your Monte Carlo agent and visualize it.

__Task 2:__  
How could you evaluate your agent?

**Answer**: We can count the number/proportion of times it won against a random and human player.

__Task 3:__  
How could you improve your agent?

**Answer**: It depends how deep do you want to go in solving this problem...
- Changing the current MC algorithm implementation with a better MC method might potentially improve the performance of the agent.
- Another change would be using reinforcement-learning methods.